In [0]:
dbutils.widgets.text('env','dev')
dbutils.widgets.text('filename','')

In [0]:
%run "/Repos/s197818@8451.com/Campaign-Ad-Group-API-Patching/dev_setup"

In [0]:
all_responses = []
filename = dbutils.widgets.get('filename')
env = dbutils.widgets.get('env')
if env == 'dev':
  print('dev')
  campaign_get_endpoint,campaign_patch_endpoint,token_endpoint,client_id,client_secret = declare_dev_credentials()
  print('Done')
elif env == 'prd':
  print('prd')
  campaign_get_endpoint,campaign_patch_endpoint,token_endpoint,client_id,client_secret = declare_prd_credentials()
else:
  print('Incorrect environment entered!!!')
  exit()

In [0]:
## Get campaign details DataFrame from filename
campaigns_df = read_campaign_patch_file(filename)

In [0]:
## Values that are going to get patched
campaigns_df.display()

In [0]:
path_list, value_list, op_list, campaign_ids  = create_json_variables(campaigns_df)

In [0]:
## Generate headers for authenticate user
headers = generate_headers(client_id,client_secret,token_endpoint)

In [0]:
def get_request(campaign_id):
  response = requests.get(campaign_get_endpoint + campaign_id , headers=headers)
  if response.status_code == 200:
    print('Successful Request!')
    return response
  else:
    print(f'Patching Failed with {response.status_code}')
    return ""

In [0]:
def patch_request(campaign_id,payload):
  response = requests.patch(campaign_patch_endpoint + campaign_id, headers=headers,json=payload)
  if response.status_code == 204:
    print(f'Successfully Patched: {campaign_id} + {payload}')
  else:
    print(f'Patching Failed with {response.status_code} \n {payload}')

In [0]:
## driver loop, iterate over each row data of csv file
for i in range(len(value_list)):
  payload = create_payload(value_list[i],op_list[i],path_list[i])
  patch_request(campaign_ids[i],payload)